In [37]:
import pdb
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

import numpy as np

import io
import os
import json

# https://github.com/bolaurent/zuora_restful_python
from zuora_restful_python.zuora import Zuora

ZUORA_CONFIG_SANDBOX = json.load(open(os.path.expanduser('~') + '/.zuora-sandbox-config.json'))


zuora_sandbox = Zuora(ZUORA_CONFIG_SANDBOX['user'],ZUORA_CONFIG_SANDBOX['password'],endpoint='sandbox')


FIELDS = [
    'RatePlanCharge.DTCV'
    ,'RecognizedRevenueAccountingCode.Name'
    ,'Subscription.Name'
    ,'Subscription.SubscriptionStartDate'
]

# given a date like '2017-01-02', return a period name like '2017-01-01'
def convert_date_to_periodname(d):
    return '{}-{}'.format(d[0:4],d[5:7])

# query rateplancharges and return a pandas dataframe, indexed on subscription name
def read_rateplancharges(zuora):
    q = """
        SELECT {} FROM RatePlanCharge 
        WHERE Subscription.SubscriptionStartDate >= '2017-04-01'
        AND Subscription.SubscriptionStartDate < '2017-07-01'
        """.format(','.join(FIELDS))
    csvData = zuora.query_export(q)
    df = pd.read_csv(io.StringIO(csvData), index_col=['Subscription.Name'])
    
    return df

# query rateplancharges from zuora
rateplancharges = read_rateplancharges(zuora_sandbox)


# create a column that shows the fiscal period, based on the subscription start date
rateplancharges['Month'] = rateplancharges['Subscription.SubscriptionStartDate']\
                            .apply(convert_date_to_periodname)

# pivot the table to show total revenue by account and period
rateplancharges = pd.pivot_table(rateplancharges, \
                                 index=['RecognizedRevenueAccountingCode.Name'], \
                                 columns=['Month'], \
                                 aggfunc=np.sum, \
                                 values=['RatePlanCharge.DTCV'])

# Obfuscate the data for publication
rateplancharges = rateplancharges.applymap(lambda x: np.random.rand() * 100000000)


# print it in a nice table
rateplancharges




RatePlanCharge.DTCV                \
Month                                            2017-04       2017-05   
RecognizedRevenueAccountingCode.Name                                     
Incidental Revenue                         26,402,922.55 83,196,242.30   
Licensing Revenue                          63,352,357.32 71,804,405.30   
Marketplace Revenue                        96,989,424.51 80,350,394.33   
Operations Revenue                         33,944,293.01 90,281,017.00   
Professional Services Revenue              39,813,175.79 44,559,213.03   
Support Revenue                            34,287,860.92 14,779,388.35   

                                                    
Month                                      2017-06  
RecognizedRevenueAccountingCode.Name                
Incidental Revenue                    6,970,661.91  
Licensing Revenue                    31,998,166.46  
Marketplace Revenue                  84,706,993.21  
Operations Revenue                   29,572,991.53  
Professional Services Revenue        72,378,496.83  
Support Revenue                      96,022,124.11